1. Importação das Bibliotecas

In [1]:
import pandas as pd
import unicodedata
from joblib import load, dump

2. Função para Normalização de Textos

In [2]:
# Função utilitária para normalizar textos
def normalizar_texto(s):
    return unicodedata.normalize('NFKD', s.strip().upper()).encode('ASCII', 'ignore').decode('utf-8') if isinstance(s, str) else s

3. Carregamento dos DataFrames

In [3]:
# === Carregamento dos DataFrames ===
categorias = load("../../util/data/df_categorias.z")
produtos = load("../../util/data/df_produtos.z")
unidades = load("../../util/data/df_unidades.z")
vendas = load("../../util/data/df_vendas.z")

4. Tratamento de Dados - Categorias

In [22]:
# === tratamento - categorias ===
print("\nantes - categorias:")
print(categorias.info())
print(categorias.head())

if not categorias.empty:
    # renomear todas as colunas para letras minúsculas
    categorias.columns = [col.lower() for col in categorias.columns]

    # padronizar o texto da descricao para minúsculas e remover espaços
    categorias['descricao'] = categorias['descricao'].apply(normalizar_texto)
    
    # remover duplicidades na descricao
    categorias.drop_duplicates(subset='descricao', keep='first', inplace=True)
    categorias.reset_index(drop=True, inplace=True)

    # salvar dataframe tratado
    dump(categorias, "../../util/data/df_categorias.z")

    print("\ndepois - categorias:")
    print(categorias.info())
    print(categorias.head())
    print("\ncategorias tratadas e salvas.")
else:
    print("\ndataframe de categorias está vazio.")




antes - categorias:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         50 non-null     int64 
 1   descricao  50 non-null     object
dtypes: int64(1), object(1)
memory usage: 932.0+ bytes
None
    id                  descricao
0   41                 COMPUTADOR
1   64                      PASTA
2  342                     CANETA
3   51                   PRESENTE
4  346  MATERIAL DE USO E CONSUMO

depois - categorias:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         50 non-null     int64 
 1   descricao  50 non-null     object
dtypes: int64(1), object(1)
memory usage: 932.0+ bytes
None
    id                  descricao
0   41                 COMPUTADOR
1   64                      PASTA
2  342         

5. Tratamento de Dados - Produtos

In [23]:
# === tratamento - produtos ===
print("\nantes - produtos:")
print(produtos.info())
print(produtos.head())

if not produtos.empty:
    # renomear todas as colunas para letras minúsculas
    produtos.columns = [col.lower() for col in produtos.columns]

    # justificativa: converter 'codigo_categoria' para inteiro para garantir a integridade dos dados.
    produtos['codigo_categoria'] = produtos['codigo_categoria'].astype('Int64')

    # justificativa: a coluna 'classificacao' não é relevante para a análise de estoque.
    produtos.drop(columns=['classificacao', 'codigo_barras'], inplace=True, errors='ignore')

    # justificativa: padronizar a unidade de medida para consistência na análise.
    produtos['unidade'] = produtos['unidade'].replace({'UNIDADE': 'UN', 'CAIXA': 'CX'}).fillna('UN').apply(normalizar_texto)

    # justificativa: remover produtos com preço de venda ausente ou inválido, pois são essenciais para cálculos de valor.
    produtos = produtos[produtos['preco_venda'].notna() & (produtos['preco_venda'] > 0)]

    # justificativa: padronizar todas as strings para minúsculo para facilitar a análise e evitar problemas de case-sensitivity.
    for col in produtos.select_dtypes(include='object').columns:
       produtos[col] = produtos[col].map(normalizar_texto)

    # salvar dataframe tratado
    dump(produtos, "../../util/data/df_produtos.z")
    
    print("\ndepois - produtos:")
    print(produtos.info())
    print(produtos.head())
    print("\nprodutos tratados e salvos.")
else:
    print("\ndataframe de produtos está vazio.")



antes - produtos:
<class 'pandas.core.frame.DataFrame'>
Index: 9199 entries, 1 to 9590
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                9198 non-null   object 
 1   descricao         9199 non-null   object 
 2   unidade           9199 non-null   object 
 3   marca             9195 non-null   object 
 4   codigo_categoria  9196 non-null   float64
 5   categoria         9196 non-null   object 
 6   preco_venda       9199 non-null   float64
dtypes: float64(2), object(5)
memory usage: 574.9+ KB
None
       id          descricao unidade marca  codigo_categoria categoria  \
1  194412   CANETA REF 35633      UN   BIC             342.0    CANETA   
3  006467    CANETA REF 5968      UN   BIC             342.0    CANETA   
4  004559    CANETA REF 4092      UN   BIC             342.0    CANETA   
5  006654  ENVELOPE REF 6152      UN    3M               9.0  ENVELOPE   
6  005543    CANETA REF 5066

6. Tratamento de Dados - Unidades

In [24]:
# === tratamento - unidades ===
print("\nantes - unidades:")
print(unidades.info())
print(unidades.head())

if not unidades.empty:
    # renomear todas as colunas para letras minúsculas
    unidades.columns = [col.lower() for col in unidades.columns]

    # padronizar o texto das colunas para garantir consistência e facilitar futuras análises ou joins
    for col in ['nome', 'cidade', 'estado']:
        if col in unidades.columns:
            unidades[col] = unidades[col].apply(normalizar_texto)

    # renomear colunas para descrições mais claras e intuitivas
    unidades.rename(columns={
        'id': 'id',
        'unidades': 'codigo_unidade',
        'cidade': 'nome_cidade',
        'estado': 'unidade_operante'
    }, inplace=True)

    # remover linhas duplicadas para garantir unicidade
    unidades.drop_duplicates(inplace=True)
    unidades.reset_index(drop=True, inplace=True)

    # salvar dataframe tratado
    dump(unidades, "../../util/data/df_unidades.z")

    print("\ndepois - unidades:")
    print(unidades.info())
    print(unidades.head())
    print("\nunidades tratadas e salvas.")
else:
    print("\ndataframe de unidades está vazio.")



antes - unidades:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             3 non-null      int64 
 1   nome_fantasia  3 non-null      object
 2   status         3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes
None
      id    nome_fantasia status
0  10003   L03 - SALVADOR    sim
1  10002     L02 - SOBRAL    sim
2  10001  L01 - FORTALEZA    sim

depois - unidades:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             3 non-null      int64 
 1   nome_fantasia  3 non-null      object
 2   status         3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes
None
      id    nome_fantasia status
0  10003   L03 - SALVADOR    sim
1  10002     

7. Tratamento de Dados - Vendas

In [26]:
# === tratamento - vendas ===
print("\nantes - vendas:")
print(vendas.info())
print(vendas.head())

if not vendas.empty:
    # renomear todas as colunas para minúsculas (opcional, mas recomendado)
    vendas.columns = [col.lower() for col in vendas.columns]

    # remover a coluna 'codigo_empregado', foco no estoque, não no desempenho
    vendas.drop(columns=["codigo_empregado"], errors="ignore", inplace=True)

    # remover colunas com mais de 80% de valores ausentes
    cols_to_drop = vendas.columns[vendas.isna().mean() > 0.8]
    vendas.drop(columns=cols_to_drop, inplace=True, errors='ignore')

    # preencher valores ausentes na coluna 'data_venda' com 'ausente'
    if 'data_venda' in vendas.columns:
        vendas['data_venda'] = vendas['data_venda'].fillna('ausente')
    else:
        print("a coluna 'data_venda' não foi encontrada no dataframe de vendas.")

    # preencher valores ausentes nas colunas numéricas 'quantidade' e 'valor_total' com 0
    for col in ['quantidade', 'valor_total']:
        if col in vendas.columns:
            vendas[col] = vendas[col].fillna(0)

    # padronizar todas as strings para minúsculo para garantir consistência
    for col in vendas.select_dtypes(include='object').columns:
        vendas[col] = vendas[col].map(normalizar_texto)

    # remover linhas duplicadas para garantir unicidade
    vendas.drop_duplicates(inplace=True)
    vendas.reset_index(drop=True, inplace=True)

    # salvar dataframe tratado
    dump(vendas, "../../util/data/df_vendas.z")

    print("\ndepois - vendas:")
    print(vendas.info())
    print(vendas.head())
    print("\nvendas tratadas e salvas.")
else:
    print("\ndataframe de vendas está vazio.")




antes - vendas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10418 entries, 0 to 10417
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   codigo_empresa    10418 non-null  int64  
 1   tipo_pedido       10418 non-null  object 
 2   centro_resultado  10418 non-null  int64  
 3   documento         10418 non-null  int64  
 4   codigo_cliente    10418 non-null  int64  
 5   data_emissao      10418 non-null  object 
 6   hora_emissao      10418 non-null  object 
 7   data_fechamento   10418 non-null  object 
 8   hora_fechamento   10418 non-null  object 
 9   data_faturamento  10408 non-null  object 
 10  codigo_produto    10418 non-null  int64  
 11  quantidade        10418 non-null  float64
 12  total             10418 non-null  float64
 13  custo_medio       10418 non-null  float64
 14  valor_desconto    10418 non-null  float64
dtypes: float64(4), int64(5), object(6)
memory usage: 1.2+ MB
None
   codig